In [2]:
import pickle

# Specify the path to your .p file
file_path = "/Users/tianyichen/Desktop/Research /PhDresearch/Twitters/2024-06-10-tweets-with-text.p"

# Open the file in binary mode
with open(file_path, "rb") as file:
    # Load the data from the file
    data = pickle.load(file)


In [3]:
data.shape

(2904271, 11)